## Read file

In [ ]:
pdf_raw = None
with open("challenge.pdf", "rb") as data:
    pdf_raw = data.read().strip()

##  convert PDF to serialized JSON object

Fileformat documentation for PDF v1.5
https://docs.fileformat.com/pdf/

In [ ]:
import sys

PDF = []

with open("challenge.pdf", "rb") as f:
    data = f.read()

tag_obj_start = b"obj\n"
tag_obj_end   = b"endobj\n"
tag_header_start = b"\x3c\x3c"
tag_header_end    = b"\x3e\x3e"
tag_stream_start = b"\x06stream"
tag_stream_end   = b"\nendstream"

header_tags = {b"/Length":int, b"/Filter":"tag", 
               b"/FlateDecode":"", b"/Type":"tag", 
               b"/Contents":list, b"/Resources":list, 
               b"/MediaBox":list, b"/Parent":list, 
               b"/Group":list}

#Finds the first tag
#@return: bytestring on successs
#@return: None on error
def find_next_tag( pdf_index_start=0 ):
    tag_index_start = pdf_raw[pdf_index_start:].find( tag_obj_start )
    if tag_index_start == -1:  #find returns -1 on failure
        return None, -1
    tag_index_start += pdf_index_start   #absolute index
    
    tag_index_end = pdf_raw[tag_index_start+len(tag_start):].find( tag_obj_end )
    if tag_index_end == -1:  #find returns -1 on failure
        return None, -1
    tag_index_end += tag_index_start + len(tag_end)  #absolute index
    
    return pdf_raw[tag_index_start:tag_index_end], tag_index_end+len(tag_end)


def parse_pdf_element(element):
    i = element.find(b"\x20")
    k = element[:i].strip()
    v = element[i:].strip()
    
    print("\t", k, " - ", v)
    t = str
    if k in header_tags.keys():
        if header_tags[k] == int:
            v = int(v)
        elif header_tags[k] == list:
            if v[0] == 91:
                v = v[1:-1]
            a = v.split(b" ")
            v = []
            for b in a:
                if b.isinteger():
                    v.append( int(b) )
                else:
                    v.append( b )
        elif header_tags[k] == "tag":
            pass
        else:
            v = None
        
    print("\t", k, " - ", v)
    print()
    return k, v


def parse_pdf_element_struct(pdf_raw_element, pdf_search_index):
    print( pdf_search_index )
    print( len(pdf_raw_element) )
    print()
    
    HEAD = None
    index_head_start = 5
    if pdf_raw_element[3:5] != tag_header_start:
        index_head_start = pdf_raw_element.find(tag_header_start)+len(tag_header_start)
    index_head_end   = pdf_raw_element.find(tag_header_end)
    
    pdf_raw_head = pdf_raw_element[index_head_start : index_head_end]
    
    head = {}
    for element in pdf_raw_head.strip().split(b"\n"):
        key,val = parse_pdf_element( element )
        head.update({key : val })
        
    
    

x = 0
print(pdf_raw[0:90])
pdf_search_index = 0
while pdf_search_index != -1 and pdf_search_index < len(pdf_raw):
    pdf_search_index_last = pdf_search_index
    pdf_raw_element, pdf_search_index = find_next_tag(pdf_search_index)
    if pdf_search_index == -1:
        print( "pdf_search_index returned -1, starting at index: " + pdf_search_index)
        break
    print("#"*80)
    
    pdf_object = parse_pdf_element_struct( pdf_raw_element, pdf_search_index )
    x+=1
    if x > 2:
        break

## Extract ruby service

In [ ]:
#out = open("ruby_pdf.txt","w")
#for c in HEX.replace("\n","").replace("  "," ").strip().split(" "):
#    out.write( chr( int(c,16) ) )
#out.close()